Parts of this code were adapted from Paredes et al., 2022 and certain parameter values were taken from Serino et al., 2015 or Magosso et al., 2010b

In [1]:
import pylab
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
from scipy.optimize import curve_fit
from scipy.optimize import minimize
from scipy.optimize import differential_evolution
from mpl_toolkits.axes_grid1 import make_axes_locatable
import string
from scipy.special import logsumexp

/tmp/ipykernel_699/1247435164.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# Network structure
Variables that define the structure of the network model in the following terms:
- dimensions of the encoded space, 
- number of neurons, 
- receptive fields (RF) centres.

In [2]:
## Tactile area
# Tactile area dimensions (cm)
xt = 20
yt = 10

# Tactile neurons (count)
Mt = 40  # Mt = 40 in Magosso2010b
Nt = 20  # Nt = 20 in Magosso2010b

# Tactile RF centres (cm)
xrft = np.arange(1, Mt + 1) * 0.5 - 0.25
yrft = np.arange(1, Nt + 1) * 0.5 - 0.25

## Auditory area
# Auditory area dimensions (cm)
xa = 200
ya = 30

# Auditory neurons (count)
Ma = 20
Na = 3

# Auditory RF centres (cm)
xrfa = np.arange(1, Ma + 1) * 10 - 5
yrfa = np.arange(1, Na + 1) * 10 - 15

## Visual area
# Visual area dimensions (cm)
xv = 100
yv = 15

xtool = 85  # or 95

# Visual neurons (count)
Mv = 100
Nv = 15

# Visual RF centres (cm)
xrfv = np.arange(1, Mv + 1) - 0.5  # As per Magosso2010b
yrfv = np.arange(1, Nv + 1) - 3  # As per Magosso2010b

dx = 0.2
dy = 0.2

# Network fixed parameters
Variables that define the values of the parameters that remain fixed in our study. These parameters are related to: 
- the RF of the unisensory neurons,
- the activation functions of both unisensory and multisensory neurons, 
- the visual and tactile stimuli that is administered during the experiment simulation and training simulation.  

In [3]:
### Model fixed parameters for simulation/experiment setting
# Question for Renato - why is ya_0 = 5, rather than 15?

## Unisensory receptive fields
phit_0 = 1  # Magosso2010b
sigmat_phi = 0.5  # Magosso2010b

phia_0 = 1  # Serino2015
sigmaa_phi = 10  # Serino2015

phiv_0 = 1  # Magosso2010b
sigmav_phi = 1  # Magosso2010b

## Neuronal activity

# Tactile neurons
fmin_t = -0.12
fmax_t = 1
qc_t = 19.43
r_t = 0.34

# Auditory neurons
fmin_a = -0.12
fmax_a = 1
qc_a = 19.43
r_a = 0.34

# Visual neurons
fmin_v = -0.12
fmax_v = 1
qc_v = 19.43
r_v = 0.34

# Multisensory neuron
fmin_m = 0
fmax_m = 1
qc_m = 19.43
r_m = 0.34

tau = 20

## External stimuli - Experimental Setting
# Tactile stimuli
It_0_exp = 2.5
sigt_exp = 0.3
yt_0_exp = 5  # cm
xt_0_exp = 10  # cm

# Auditory stimuli
Ia_0_exp = 3.6
siga_exp = 0.3
ya_0_exp = 5  # cm
xa_0_exp = 100  # cm

# Visual Stimuli:
# Participant is blindfolded during experimental setting - no visual stimulus

# Gaussian noise

signoise_t = 0
signoise_a = 0

stdnoise = 1

## External stimuli - Training Setting
# Tactile stimuli
It_0_tr = 0.56
sigtx_tr = 0.75
sigty_tr = 1.25

# Auditory stimuli
Ia_0_tr = 0.0125
sigax_tr = 3.75
sigay_tr = 6

# Visual Stimuli:
Iv_0_tr = 0.121
sigvx_tr = 3.75
sigvy_tr = 6

# Gaussian noise - training auditory and visual stimuli are much more noisy than before

signoiset_t = 0  # 0.01
signoiset_a = 0  # 0.2
signoiset_v = 0  # 0.3

stdnoise = 1

# External stimuli functions

In [4]:
## External stimulus - experiment run


def stimspace_exp(s, x0, y0):
    """Compute the stimulus intensity for the unimodal space s (t/a) for a stimulus centered around x0 and y0. This stimulus is specifically related to
    the experimental setting, which exlusively utilises audio-tactile information.

    Input:
        s  (char): region of stimulation (tactile/auditory)
        x0 (int):  center point on the x axis of the stimulus
        y0 (int):  center point on the y axis of the stimulus

    Output:
        v (2D.np.array): matrix of stimulus intensity for the entire unimodal space
    """

    if s == "t":
        a = np.reshape(
            np.repeat(np.arange(0, xt + dx, dx), int(yt / dy + 1)),
            (int(xt / dx + 1), int(yt / dy + 1)),
        )
        b = np.reshape(
            np.tile(np.arange(0, yt + dx, dx), (int(xt / dx + 1))),
            (int(xt / dx + 1), int(yt / dy + 1)),
        )

        v = (
            np.ones((int(xt / dx + 1), int(yt / dy + 1))) * It_0_exp
            + np.ones((int(xt / dx + 1), int(yt / dy + 1)))
            * signoise_t
            * np.random.randn()
        ) * np.exp(
            -(
                np.square(np.ones((int(xt / dx + 1), int(yt / dy + 1))) * x0 - a)
                + np.square(np.ones((int(xt / dx + 1), int(yt / dy + 1))) * y0 - b)
            )
            / (2 * np.square(sigt_exp))
        )
    else:
        a = np.reshape(
            np.repeat(np.arange(0, xa + dx, dx), int(ya / dy + 1)),
            (int(xa / dx + 1), int(ya / dy + 1)),
        )
        b = np.reshape(
            np.tile(np.arange(0, ya + dx, dx), (int(xa / dx + 1))),
            (int(xa / dx + 1), int(ya / dy + 1)),
        )

        v = (
            np.ones((int(xa / dx + 1), int(ya / dy + 1))) * Ia_0_exp
            + np.ones((int(xa / dx + 1), int(ya / dy + 1)))
            * signoise_a
            * np.random.randn()
        ) * np.exp(
            -(
                np.square(np.ones((int(xa / dx + 1), int(ya / dy + 1))) * x0 - a)
                + np.square(np.ones((int(xa / dx + 1), int(ya / dy + 1))) * y0 - b)
            )
            / (2 * np.square(siga_exp))
        )

    return v

# Receptive fields functions

In [5]:
# RF formula


def phi(s, x, y):
    """Obtain the RF of the unimodality s (t/a/v) for a given set of spatial coordinates.

    Input:
        s (char): unimodality (t/a/v)
        x (int): Spatial coordinate in the x-axis (cm).
        y (int): Spatial coordinate in the y-axis (cm).

    Output:
        phi (2D.np.array): RF of the unimodality for the given spatial coordinates.
    """

    # Build the RF matrix
    if s == "t":
        phi = np.zeros((Mt, Nt))

        # Compute the RF for the tactile space on the given spatial coordinates
        for i in range(Mt):
            for j in range(Nt):
                phi[i, j] = phit_0 * np.exp(
                    -(
                        (np.square(xrft[i] - x) + np.square(yrft[j] - y))
                        / (2 * np.square(sigmat_phi))
                    )
                )

    elif s == "a":
        phi = np.zeros((Ma, Na))

        # Compute the RF for the auditory space on the given spacial coordinates
        for i in range(Ma):
            for j in range(Na):
                phi[i, j] = phia_0 * np.exp(
                    -(
                        (np.square(xrfa[i] - x) + np.square(yrfa[j] - y))
                        / (2 * np.square(sigmaa_phi))
                    )
                )

    else:
        phi = np.zeros((Mv, Nv))

        # Compute the RF for the visual space on the given spatial coordinates
        for i in range(Mv):
            for j in range(Nv):
                phi[i][j] = phiv_0 * np.exp(
                    -(
                        (np.square(xrfv[i] - x) + np.square(yrfv[j] - y))
                        / (2 * np.square(sigmav_phi))
                    )
                )
    return phi


def PhiRF(s):
    """Compute the RF of the unisensory areas for the discreteised spatial coordinates in the auditory and tactile spaces.
    I run this once and then save the RFs as they do not change.

    Input:
        s (char): the unimodal region of interest (t/a/v)

    Output:
        Phi (4D np.array): RF of the area for the discretised spatial coordinates in the space.
    """

    # Calculate the discretised spatial coordinates
    if s == "t":
        xl = xt
        yn = yt
        Phi = np.zeros((Mt, Nt, int(xt / dx + 1), int(yt / dy + 1)))

    elif s == "a":
        xl = xa
        yn = ya
        Phi = np.zeros((Ma, Na, int(xa / dx + 1), int(ya / dy + 1)))

    else:
        xl = xv
        yn = yv
        Phi = np.zeros((Mv, Nv, int(xv / dx + 1), int(yv / dy + 1)))

    for k in range(int(xl / dx + 1)):
        for l in range(int(yn / dy + 1)):
            Phi[:, :, k, l] = phi(s, k * dx, l * dy)

    return Phi

In [7]:
# Uncomment if you do no thave access to these numpy objects already

# Phit = PhiRF("t")
# Phia = PhiRF("a")
# Phiv = PhiRF("v")

# np.save("drafts/Phit.npy", Phit)
# np.save("drafts/Phia.npy", Phia)
# np.save("drafts/Phiv.npy", Phiv)

In [ ]:
Phit = np.load("drafts/Phit.npy")
Phia = np.load("drafts/Phia.npy")
Phiv = np.load("drafts/Phiv.npy")

In [ ]:
def PHI_expa(s, x0, y0):
    """This function computes the external input to the unimodal area "s" based on a stimulus centered around x0 and y0
    Input:
        s (char): unimodality (a/t)
        x0 (int): center of the external stimulus on the x axis
        y0 (int): center of the external stimulus on the y axis

    Assumed Inputs:
        Phit (4D.np.array): a matrix of each tactile neuron's receptive field
        Phia (4D.np.array): a matrix of each auditory neuron's receptive field
    """

    PHI = np.sum(np.multiply(Phia, stimspace_exp(s, x0, y0)), (3, 2))

    return PHI


def PHI_expt(s, x0, y0):
    """This function computes the external input to the unimodal area "s" based on a stimulus centered around x0 and y0
    Input:
        s (char): unimodality (a/t)
        x0 (int): center of the external stimulus on the x axis
        y0 (int): center of the external stimulus on the y axis

    Assumed Inputs:
        Phit (4D.np.array): a matrix of each tactile neuron's receptive field
        Phia (4D.np.array): a matrix of each auditory neuron's receptive field
    """

    PHI = np.sum(np.multiply(Phit, stimspace_exp(s, x0, y0)), (3, 2))

    return PHI

# Synapses functions

In [ ]:
def Lw(
    Lex_t,
    Lin_t,
    sigmaex_t,
    sigmain_t,
    Lex_a,
    Lin_a,
    sigmaex_a,
    sigmain_a,
    Lex_v,
    Lin_v,
    sigmaex_v,
    sigmain_v,
):
    """Compute the matrix of lateral connections (akin to a Mexican Hat Function). Still have not decided on the border effects.

    Input:
        Lex_t (int): the excitatory amplitude for the tactile lateral synapses
        sigmaex_t (int): the excitatory spread for the tactile lateral synapses
        Lin_t (int): the inhibitory amplitude for the tactile synapses
        sigmain_t (int): the inhibitory spread for the tactile synapses

        Lex_a (int): the excitatory amplitude for the auditory lateral synapses
        sigmaex_a (int): the excitatory spread for the auditory lateral synapses
        Lin_a (int): the inhibitory amplitude for the auditory synapses
        sigmain_a (int): the inhibitory spread for the auditory synapses

        Lex_v (int): the excitatory amplitude for the visual lateral synapses
        sigmaex_v (int): the excitatory spread for the visual lateral synapses
        Lin_v (int): the inhibitory amplitude for the visual synapses
        sigmain_v (int): the inhibitory spread for the visual synapses

    Output:
        Lt (4D.np.array): the map of tactile lateral connections for each neuron (each ij position has a matrix detailing its connections with all neurons)
        La (4D.np.array): the map of auditory lateral connections for each neuron (each ij position has a matrix detailing its connections with all neurons)
        Lv (4D.np.array): the map of visual lateral connections for each neuron (each ij position has a matrix detailing its connections with all neurons)
    """
    # Tactile recurrent connections

    # calculate Dx and Dy using matrix operations

    Lt = Lex_t * np.exp(
        -(
            np.square(xrft[:, None, None, None] - xrft[None, None, :, None])
            + np.square(yrft[None, :, None, None] - yrft[None, None, None, :])
        )
        / (2 * sigmaex_t**2)
    ) - Lin_t * np.exp(
        -(
            np.square(xrft[:, None, None, None] - xrft[None, None, :, None])
            + np.square(yrft[None, :, None, None] - yrft[None, None, None, :])
        )
        / (2 * sigmain_t**2)
    )
    La = Lex_a * np.exp(
        -(
            np.square(xrfa[:, None, None, None] - xrfa[None, None, :, None])
            + np.square(yrfa[None, :, None, None] - yrfa[None, None, None, :])
        )
        / (2 * sigmaex_a**2)
    ) - Lin_a * np.exp(
        -(
            np.square(xrfa[:, None, None, None] - xrfa[None, None, :, None])
            + np.square(yrfa[None, :, None, None] - yrfa[None, None, None, :])
        )
        / (2 * sigmain_a**2)
    )
    Lv = Lex_v * np.exp(
        -(
            np.square(xrfv[:, None, None, None] - xrfv[None, None, :, None])
            + np.square(yrfv[None, :, None, None] - yrfv[None, None, None, :])
        )
        / (2 * sigmaex_v**2)
    ) - Lin_v * np.exp(
        -(
            np.square(xrfv[:, None, None, None] - xrfv[None, None, :, None])
            + np.square(yrfv[None, :, None, None] - yrfv[None, None, None, :])
        )
        / (2 * sigmain_v**2)
    )

    Lt = np.where(Lt == np.max(Lt), 0, Lt)
    La = np.where(La == np.max(La), 0, La)
    Lv = np.where(Lv == np.max(Lv), 0, Lv)

    return Lt, La, Lv


def FwFb(Wt_0, Wa_0, Wv_0, Bt_0, Ba_0, Bv_0, k1, k2, lim, alpha):
    """Feedforward and feedback synaptic weights of the unisensory neurons.

    Args:
        Wt_0 (int): Maximum value of the feedforward synapses in the tactile area.
        Wa_0 (int): Maximum value of the feedforward synapses in the auditory area.
        Wv_0 (int): Maximum value of the feedforward synapses in the visual area.

        Bt_0 (int): Maximum value of the feedback synapses in the tactile area.
        Ba_0 (int): Maximum value of the feedback synapses in the auditory area.
        Bv_0 (int): Maximum value of the feedback synapses in the visual area.

        k1 (int): Fast decay rate.
        k2 (int): Slow decay rate.
        lim (int): Boundary of the space on and near the hand.
        alpha (int): Relative amplitude of each exponential.

    Returns:
        Wt (2D np.array): Feedforward synaptic weights in the tactile area.
        Wa (2D np.array): Feedforward synaptic weights in the auditory area.
        Wv (2D np.array): Feedforward synaptic weights in the visual area.

        Bt (2D np.array): Feedback synaptic weights in the tactile area.
        Ba (2D np.array): Feedback synaptic weights in the auditory area.
        Bv (2D np.array): Feedback synaptic weights in the visual area.
    """

    # Build the feedforward and feedback synapses matrices

    Bt = np.ones((Mt, Nt)) * Bt_0  # tactile feedback synapses identical
    Wt = np.ones((Mt, Nt)) * Wt_0  # tactile feedback synapses identical

    Ba = np.zeros((Ma, Na))
    Wa = np.zeros((Ma, Na))

    Bv = np.zeros((Mv, Nv))
    Wv = np.zeros((Mv, Nv))

    # Compute the feedforward and feedback synapses in the auditory area
    for i in range(Ma):
        for j in range(Na):
            if xrfa[i] < lim:
                D = 0
            else:
                D = np.linalg.norm(xrfa[i] - lim)

            Ba[i, j] = alpha * Ba_0 * np.exp(-D / k1) + (1 - alpha) * Ba_0 * np.exp(
                -D / k2
            )
            Wa[i, j] = alpha * Wa_0 * np.exp(-D / k1) + (1 - alpha) * Wa_0 * np.exp(
                -D / k2
            )

    # Compute the feedforward and feedback synapses in the visual area
    for i in range(Mv):
        for j in range(Nv):
            if xrfv[i] < lim:
                D = 0
            else:
                D = np.linalg.norm(xrfv[i] - lim)

            Bv[i, j] = alpha * Bv_0 * np.exp(-D / k1) + (1 - alpha) * Bv_0 * np.exp(
                -D / k2
            )
            Wv[i, j] = alpha * Wv_0 * np.exp(-D / k1) + (1 - alpha) * Wv_0 * np.exp(
                -D / k2
            )

    return Wt, Wa, Wv, Bt, Ba, Bv

# Cross Modal Synapses

In [ ]:
def crossmodal(at, sigat, vt, sigvt, av, sigav):
    """Create the crossmodal synapses across all unimodal regions. These synapses act similarly to the lateral connections.
    They mimic a mexican hat function, having an excitatory peak, followed by  inhibitory surroundings.

    Input:
    at (int): strength of the excitatory Gaussian for the audiotactile connections
    sigat (int): spread of the excitatory Gaussian for the audiotactile connections

    av (int): strength of the excitatory Gaussian for the audiovisual connections
    sigav (int): spread of the excitatory Gaussian for the audiovisual connections

    vt (int): strength of the excitatory Gaussian for the visuotactile connections
    sigvt (int): spread of the excitatory Gaussian for the visuootactile connections

    Output:

    Wat (4D.np.array): a MxN matrix of auditory neurons, each containing their connections to the MxN tactile neurons (i,j,:,:); or alternatively
    a Mx N matrix of tactile neurons, each containing their connections to the MxN auditory neurons (:,:,i,j)
    Wav (4D.np.array): a MxN matrix of visual neurons, each containing their connections to the MxN auditory neurons (i,j,:,:); or alternatively
    a Mx N matrix of auditory neurons, each containing their connections to the MxN visual neurons (:,:,i,j)
    Wvt (4D.np.array): a MxN matrix of visual neurons, each containing their connections to the MxN tactile neurons (i,j,:,:); or alternatively
    a Mx N matrix of tactile neurons, each containing their connections to the MxN visual neurons (:,:,i,j)
    """

    dx = xrfa.reshape(Ma, 1, 1, 1) - xrft.reshape(1, 1, Mt, 1)
    dy = yrfa.reshape(1, Na, 1, 1) - yrft.reshape(1, 1, 1, Nt)
    Wat = at * np.exp(-((np.square(dx) + np.square(dy)) / (2 * np.square(sigat))))

    dx = xrfv.reshape(Mv, 1, 1, 1) - xrft.reshape(1, 1, Mt, 1)
    dy = yrfv.reshape(1, Nv, 1, 1) - yrft.reshape(1, 1, 1, Nt)
    Wvt = vt * np.exp(-((np.square(dx) + np.square(dy)) / (2 * np.square(sigvt))))

    dx = xrfa.reshape(Ma, 1, 1, 1) - xrfv.reshape(1, 1, Mv, 1)
    dy = yrfa.reshape(1, Na, 1, 1) - yrfv.reshape(1, 1, 1, Nv)
    Wav = av * np.exp(-((np.square(dx) + np.square(dy)) / (2 * np.square(sigav))))

    return Wat, Wvt, Wav

# Neural activity functions

In [ ]:
def neural(qt, qa, qv, qm, Lt, La, Lv, zt, za, zv, Wat, Wvt, Wav, zm, Bt, Ba, Bv):
    """Compute the rates (z) as the sigmoid of the dynamic state variable

    Input:
        s (char): unimodality (a/t/v)
        q (2D.np.array): state variable

    Assumed Inputs:
        fmin_t (int): lower boundary of the tactile sigmoid function
        fmax_t (int): upper boundary of the tactile sigmoid function
        qc_t (int): central point of the tactile sigmoid function
        r_t (int): slope of the tactile sigmoid curve at the central point qc_t

        fmin_a (int): lower boundary of the auditory sigmoid function
        fmax_a (int): upper boundary of the auditory sigmoid function
        qc_a (int): central point of the auditory sigmoid function
        r_a (int): slope of the auditory sigmoid curve at the central point qc_a

        fmin_m (int): lower boundary of the multisensory sigmoid function
        fmax_m (int): upper boundary of the multisensory sigmoid function
        qc_m (int): central point of the multisensory sigmoid function
        r_m (int): slope of the multisensory sigmoid curve at the central point qc_m

    Output:
        psi (2D.np.array) the rates for the unimodality s
    """

    # Cross-Modal Inputs

    psit = (
        np.ones((Mt, Nt)) * fmin_t
        + fmax_t * np.exp((qt - np.ones((Mt, Nt)) * qc_t) * r_t)
    ) / (np.ones((Mt, Nt)) + np.exp((qt - np.ones((Mt, Nt)) * qc_t) * r_t))
    psia = (
        np.ones((Ma, Na)) * fmin_a
        + fmax_a * np.exp((qa - np.ones((Ma, Na)) * qc_a) * r_a)
    ) / (np.ones((Ma, Na)) + np.exp((qa - np.ones((Ma, Na)) * qc_a) * r_a))
    psiv = (
        np.ones((Mv, Nv)) * fmin_v
        + fmax_v * np.exp((qv - np.ones((Mv, Nv)) * qc_v) * r_v)
    ) / (np.ones((Mv, Nv)) + np.exp((qv - np.ones((Mv, Nv)) * qc_v) * r_v))

    psim = (fmin_m + fmax_m * np.exp((qm - qc_m) * r_m)) / (
        1 + np.exp((qm - qc_m) * r_m)
    )

    LT = np.einsum("hkij, ij -> hk", Lt, zt)
    LA = np.einsum("hkij, ij -> hk", La, za)
    LV = np.einsum("hkij, ij -> hk", Lv, zv)
    AT = np.einsum("ijhk, ij -> hk", Wat, za)
    AV = np.einsum("ijhk, ij -> hk", Wav, za)
    VT = np.einsum("ijhk, ij -> hk", Wvt, zv)
    VA = np.einsum("hkij, ij -> hk", Wav, zv)
    TA = np.einsum("hkij, ij -> hk", Wat, zt)
    TV = np.einsum("hkij, ij -> hk", Wvt, zt)

    BT = np.multiply(Bt, zm)
    BA = np.multiply(Ba, zm)
    BV = np.multiply(Bv, zm)

    return psit, psia, psiv, psim, LT, LA, LV, AT, AV, TA, TV, VT, VA, BT, BA, BV

# Evolutionary pruning mechanism

In [ ]:
# Built after Paredes et al., 2021


def prun(WM, pr):
    """Computes synaptic pruning according to a fixed threshold rule.

    Inout:
        WM (2D np.array/4D.np.array): Feedforward/Cross-Modal synaptic weight matrix.
        pr (number): Pruning threshold.

    Output:
        newM (2D np.array/4D.np.array): Pruned weight matrix.
    """

    # Copy the original matrix
    newM = np.copy(WM)

    # Prune synaptic weights below the given threshold
    newM[newM < pr] = 0

    return newM

# Audio-tactile experiment function

In [ ]:
def experiment(
    ts, T, dist, ya, Lt, La, Lv, Wt, Wa, Wv, Bt, Ba, Bv, Wat, Wvt, Wav, FWpr, CMpr
):
    """Run a PPS-assessment simulation using the following parameters
    Input:
        ts (int): the discretising timestep of the Euler integration
        T (int): the total simulation runtime
        tau (int): time constant
        dist (1D.np.array): vector of distance
        ya (int): location on the y-axis of the auditory stimulus

        Lt (4D.np.array): A matrix of shape (MxN) in which every ij position is another matrix of shape (MxN),
                          in which every hk position details the lateral connectivity of ij with hk for the tactile region
        La (4D.np.array): A matrix of shape (MxN) in which every ij position is another matrix of shape (MxN),
                          in which every hk position details the lateral connectivity of ij with hk for the auditory region
        Lv (4D.np.array): A matrix of shape (MxN) in which every ij position is another matrix of shape (MxN),
                          in which every hk position details the lateral connectivity of ij with hk for the visual region

        Wt (2D np.array): Feedforward synaptic weights in the tactile area.
        Wa (2D np.array): Feedforward synaptic weights in the auditory area.
        Wv (2D np.array): Feedforward synaptic weights in the visual area.

        Bt (2D np.array): Feedback synaptic weights in the tactile area.
        Ba (2D np.array): Feedback synaptic weights in the auditory area.
        Bv (2D np.array): Feedback synaptic weights in the visual area

        Wat (4D.np.array): a MxN matrix of auditory neurons, each containing their connections to the MxN tactile neurons (i,j,:,:); or alternatively
        a Mx N matrix of tactile neurons, each containing their connections to the MxN auditory neurons (:,:,i,j)
        Wav (4D.np.array): a MxN matrix of visual neurons, each containing their connections to the MxN auditory neurons (i,j,:,:); or alternatively
        a Mx N matrix of auditory neurons, each containing their connections to the MxN visual neurons (:,:,i,j)
        Wvt (4D.np.array): a MxN matrix of visual neurons, each containing their connections to the MxN tactile neurons (i,j,:,:); or alternatively
        a Mx N matrix of tactile neurons, each containing their connections to the MxN visual neurons (:,:,i,j)

    Output:
        zt (4D.np.array): the activation values for the tactile region for each timestep given each auditory vertical distance
        za (4D.np.array): the activation values for the auditory region for each timestep given each auditory vertical distance
        zv (4D.np.array): the activation values for the visual region for each timestep given each auditory vertical distance
        zm (2D.np.array): the activation values for the multisensory neuron for each timestep given each auditory vertical distance
        RTs (1D.np.array): the reaction times for each given distance
    """
    n = int(T / ts)  # number of iterations
    tt = ts / tau
    # Pruning of Connections

    Wa = prun(Wa, FWpr)
    Wv = prun(Wv, FWpr)

    Wat = prun(Wat, CMpr)
    Wvt = prun(Wvt, CMpr)
    Wav = prun(Wav, CMpr)

    ti = PHI_expt("t", xt / 2, yt / 2)  # tactile stimuluis

    qt = np.zeros((Mt, Nt, len(dist), n))
    ut = np.zeros((Mt, Nt, len(dist), n))
    zt = np.zeros((Mt, Nt, len(dist), n))

    qa = np.zeros((Ma, Na, len(dist), n))
    ua = np.zeros((Ma, Na, len(dist), n))
    za = np.zeros((Ma, Na, len(dist), n))

    qv = np.zeros((Mv, Nv, len(dist), n))
    uv = np.zeros((Mv, Nv, len(dist), n))
    zv = np.zeros((Mv, Nv, len(dist), n))

    qm = np.zeros((len(dist), n))
    um = np.zeros((len(dist), n))
    zm = np.zeros((len(dist), n))

    rt = np.zeros((len(dist), n))
    RTs = np.zeros(len(dist))

    for i in range(len(dist)):
        ai = PHI_expa("a", dist[i], ya)  # auditory stimulus
        for j in range(1, n):
            (
                psit,
                psia,
                psiv,
                psim,
                LT,
                LA,
                LV,
                AT,
                AV,
                TA,
                TV,
                VT,
                VA,
                BT,
                BA,
                BV,
            ) = neural(
                qt[:, :, i, j - 1],
                qa[:, :, i, j - 1],
                qv[:, :, i, j - 1],
                qm[i, j - 1],
                Lt,
                La,
                Lv,
                zt[:, :, i, j - 1],
                za[:, :, i, j - 1],
                zv[:, :, i, j - 1],
                Wat,
                Wvt,
                Wav,
                zm[i, j - 1],
                Bt,
                Ba,
                Bv,
            )

            ut[:, :, i, j] = ti + LT + BT + AT + VT
            qt[:, :, i, j] = qt[:, :, i, j - 1] + tt * (
                -qt[:, :, i, j - 1] + ut[:, :, i, j - 1]
            )
            zt[:, :, i, j] = np.maximum(psit, 0)  # heaviside function

            ua[:, :, i, j] = ai + LA + BA + TA + VA
            qa[:, :, i, j] = qa[:, :, i, j - 1] + tt * (
                -qa[:, :, i, j - 1] + ua[:, :, i, j - 1]
            )
            za[:, :, i, j] = np.maximum(psia, 0)  # heaviside function

            uv[:, :, i, j] = LV + BV + TV + AV
            qv[:, :, i, j] = qv[:, :, i, j - 1] + tt * (
                -qv[:, :, i, j - 1] + uv[:, :, i, j - 1]
            )
            zv[:, :, i, j] = np.maximum(psiv, 0)  # heaviside function

            um[i, j] = (
                np.einsum("ij, ij -> ", Wt, zt[:, :, i, j - 1])
                + np.einsum("ij, ij -> ", Wa, za[:, :, i, j - 1])
                + np.einsum("ij, ij -> ", Wv, zv[:, :, i, j - 1])
            )
            qm[i, j] = qm[i, j - 1] + tt * (-qm[i, j - 1] + um[i, j - 1])
            zm[i, j] = np.maximum(psim, 0)  # heaviside function

            rt[i, j] = np.any(zt[:, :, i, j - 1] > (0.9))

        # Compute the RT for this distance

        RTs[i] = np.argmax(rt[i, :]) * ts  # * 3 + 60

    return zt, za, zm, zv, RTs

# Training Input Functions

In [ ]:
def stim_tr(s, x, y, x0, y0, sigav_x):
    """Compute the training stimuli for one point for modality s (a/t/v)

    Input:
    s (char): the modality (auditory/tactile/visual)
    x (int): x coordinate
    y (int): y coordinate

    Output:

    I (int): the stimulus intensity
    """

    sigax_tr = sigav_x
    sigvx_tr = sigav_x

    if s == "t":
        # if ((3 < y < 7) and ((5 < x < 10))): I = It_0_tr
        # else: I = 0
        I = (It_0_tr + signoiset_t * np.random.randn()) * np.exp(
            -((np.square(x0 - x)) / (2 * np.square(sigtx_tr)))
            - ((np.square(y0 - y)) / (2 * np.square(sigty_tr)))
        )

    elif s == "a":
        I = (Ia_0_tr + signoiset_a * np.random.randn()) * np.exp(
            -((np.square(x0 - x)) / (2 * np.square(sigax_tr)))
            - ((np.square(y0 - y)) / (2 * np.square(sigay_tr)))
        )

    else:
        I = (Iv_0_tr + signoiset_v * np.random.randn()) * np.exp(
            -((np.square(x0 - x)) / (2 * np.square(sigvx_tr)))
            - ((np.square(y0 - y)) / (2 * np.square(sigvy_tr)))
        )

    return I


def stimspace_tr(s, x0, y0, sigav_x):
    """Compute the stimulus intensity for the unimodal space s (t/a) for a stimulus centered around x0 and y0

    Input:
        s  (char): region of stimulation (tactile/auditory)
        x0 (int): vertical center of stimulus
        y0 (int): horizontal center of stimulus

    Output:
        v (2D.np.array): matrix of stimulus intensity for the entire unimodal space
    """

    sigax_tr = sigav_x
    sigvx_tr = sigav_x
    sigtx_tr = sigav_x

    if s == "t":
        v = np.zeros((int(xt / dx + 1), int(yt / dy + 1)))
        for x in range(int(xt / dx + 1)):
            for y in range(int(yt / dy + 1)):
                v[x, y] = stim_tr("t", x * dx, y * dy, x0, y0, sigtx_tr)

    elif s == "a":
        v = np.zeros((int(xa / dx + 1), int(ya / dy + 1)))
        for x in range(int(xa / dy + 1)):
            for y in range(int(ya / dy + 1)):
                v[x, y] = stim_tr("a", x * dx, y * dy, x0, y0, sigax_tr)
    else:
        v = np.zeros((int(xv / dx + 1), int(yv / dy + 1)))
        for x in range(int(xv / dy + 1)):
            for y in range(int(yv / dy + 1)):
                v[x, y] = stim_tr("v", x * dx, y * dy, x0, y0, sigvx_tr)

    return v


def PHI_tr(s, x0, y0, sigav_x):
    """This function computes the external input to the unimodal area "s" based on a stimulus centered around x0 and y0
    Input:
        s (char): unimodality (a/t)
        x0 (int): center of the external stimulus on the x axis
        y0 (int): center of the external stimulus on the y axis

    Assumed Inputs:
        Phit (4D.np.array): a matrix of each tactile neuron's receptive field
        Phia (4D.np.array): a matrix of each auditory neuron's receptive field
    """

    if s == "t":
        Phi = Phit
    elif s == "a":
        Phi = Phia
    else:
        Phi = Phiv

    PHI = np.multiply(Phi, stimspace_tr(s, x0, y0, sigav_x))

    PHI = np.sum(PHI, axis=3)
    PHI = np.sum(PHI, axis=2)

    return PHI

# Auditory - Visual - Tactile Training 

In [ ]:
# 100 total movements, so 100 network presentations?


def FWTraining(
    m,
    T,
    ts,
    Lt,
    La,
    Lv,
    Wt,
    Wa,
    Wv,
    Bt,
    Ba,
    Bv,
    Wat,
    Wvt,
    Wav,
    FWpr,
    rho_0,
    Wmax_a,
    Wmax_v,
    k_v,
    k_a,
    theta,
    sigav_x,
    locav_x,
):
    """Train the feedforward auditory and visual weights

    Input:

        stcount (int): number of presentations
        T (int): simulation steps (ms)
        ts (int): timestep

        Lt (4D.np.array): tactile lateral connections
        La (4D.np.array): auditory lateral connections
        Lv (4D.np.array): visual lateral connections
        Wt (2D.np.array): feedforward tactile connections
        Wa (2D.np.array): feedforward auditory connections
        Wv (2D.np.array): feedforward visual connections
        Bt (2D.np.array): feedback tactile connections
        Ba (2D.np.array): feedback auditory connections
        Bv (2D.np.array): feedback visual connections

        rho_0 (int): reinforcing factor
        Wmax_a (int): maximum value for auditory weights
        Wmax_v (int): maximum value for visual weights
        k_a (int): auditory forgetting factor
        k_v (int): visual forgetting factor
        theta (int): multisensory neuron activation thrseshold

    Output:
        Wa_tr (4D.np.array): feedfoward auditory connections for the snychronous condition
        Wv_tr (4D.np.array): feedfoward visual connections for the snychronous condition
    """
    # Initialise Weights // initially, they start off with the predefined feedforward weights // AS - Asynchronous, S - Synchronous

    n = int(T / ts)
    tt = ts / tau

    Wa = prun(Wa, FWpr)
    Wv = prun(Wv, FWpr)

    Wa_tr = np.zeros((Ma, Na, m, n))
    Wv_tr = np.zeros((Mv, Nv, m, n))

    Wa_tr[:, :, 0, 0] = Wa
    Wv_tr[:, :, 0, 0] = Wv

    qt = np.zeros((Mt, Nt, m, n))
    ut = np.zeros((Mt, Nt, m, n))
    zt = np.zeros((Mt, Nt, m, n))

    qa = np.zeros((Ma, Na, m, n))
    ua = np.zeros((Ma, Na, m, n))
    za = np.zeros((Ma, Na, m, n))

    qv = np.zeros((Mv, Nv, m, n))
    uv = np.zeros((Mv, Nv, m, n))
    zv = np.zeros((Mv, Nv, m, n))

    qm = np.zeros((m, n))
    um = np.zeros((m, n))
    zm = np.zeros((m, n))

    ti = PHI_tr("t", 10, 5, sigav_x)  # tactile stimulus
    ai = PHI_tr("a", locav_x, 5, sigav_x)  # auditory stimulus
    vi = PHI_tr("v", locav_x, 5, sigav_x)  # visual stimulus

    for i in range(m):
        # Intialise weights, either to the predefined ones, or to the ones from the previous simulation
        if i != 0:
            Wa_tr[:, :, i, 0] = Wa_tr[:, :, i - 1, -1]
            Wv_tr[:, :, i, 0] = Wv_tr[:, :, i - 1, -1]

        for j in range(1, n):
            (
                psit,
                psia,
                psiv,
                psim,
                LT,
                LA,
                LV,
                AT,
                AV,
                TA,
                TV,
                VT,
                VA,
                BT,
                BA,
                BV,
            ) = neural(
                qt[:, :, i, j - 1],
                qa[:, :, i, j - 1],
                qv[:, :, i, j - 1],
                qm[i, j - 1],
                Lt,
                La,
                Lv,
                zt[:, :, i, j - 1],
                za[:, :, i, j - 1],
                zv[:, :, i, j - 1],
                Wat,
                Wvt,
                Wav,
                zm[i, j - 1],
                Bt,
                Ba,
                Bv,
            )

            ut[:, :, i, j] = ti + LT + BT + AT + VT
            qt[:, :, i, j] = qt[:, :, i, j - 1] + tt * (
                -qt[:, :, i, j - 1] + ut[:, :, i, j - 1]
            )
            zt[:, :, i, j] = np.maximum(psit, 0)  # heaviside function

            ua[:, :, i, j] = ai + LA + BA + TA + VA
            qa[:, :, i, j] = qa[:, :, i, j - 1] + tt * (
                -qa[:, :, i, j - 1] + ua[:, :, i, j - 1]
            )
            za[:, :, i, j] = np.maximum(psia, 0)  # heaviside function

            uv[:, :, i, j] = vi + LV + BV + AV + TV
            qv[:, :, i, j] = qv[:, :, i, j - 1] + tt * (
                -qv[:, :, i, j - 1] + uv[:, :, i, j - 1]
            )
            zv[:, :, i, j] = np.maximum(psiv, 0)  # heaviside function

            qm[i, j] = qm[i, j - 1] + tt * (-qm[i, j - 1] + um[i, j - 1])
            um[i, j] = (
                np.einsum("ij, ij -> ", Wt, zt[:, :, i, j - 1])
                + np.einsum("ij, ij -> ", Wa_tr[:, :, i, j - 1], za[:, :, i, j - 1])
                + np.einsum("ij, ij -> ", Wv_tr[:, :, i, j - 1], zv[:, :, i, j - 1])
            )
            zm[i, j] = np.maximum(psim, 0)  # heaviside function

            # Auditory Feedforward Training
            dWa = (rho_0 * (Wmax_a - Wa_tr[:, :, i, j - 1])) * za[
                :, :, i, j - 1
            ] * np.maximum((zm[i, j - 1] - theta), 0) - k_a * np.maximum(
                (zm[i, j - 1] - theta), 0
            ) * (
                Wa_tr[:, :, i, j - 1] - Wa_tr[:, :, 0, 0]
            )
            Wa_tr[:, :, i, j] = Wa_tr[:, :, i, j - 1] + dWa

            # Visual Feedforward Training
            dWv = (rho_0 * (Wmax_v - Wv_tr[:, :, i, j - 1])) * zv[
                :, :, i, j - 1
            ] * np.maximum((zm[i, j - 1] - theta), 0) - k_v * np.maximum(
                (zm[i, j - 1] - theta), 0
            ) * (
                Wv_tr[:, :, i, j - 1] - Wv_tr[:, :, 0, 0]
            )
            Wv_tr[:, :, i, j] = Wv_tr[:, :, i, j - 1] + dWv

            # Tactile Training
            # dW_t = (rho_0 * (Wmax_t - WS_t[:,:, i, j - 1])) * zt[:,:,i, j - 1] * (zm[i,j-1] - theta) * np.heaviside((zm[i,j-1] - theta), 0) - k_t * (zm[i,j-1] - theta) * np.heaviside((zm[i,j-1] - theta), 0) *(WS_t[:,:, i, j - 1] - WS_t[:,:, i, 0])
            # WS_t[:,:,i,j] = WS_t[:,:,i,j - 1] + dW_t

    return Wa_tr, Wv_tr, zm, zv, za, zt

# Sigmoid fitting functions 

In [ ]:
def RTsig(distances, cp, slope):
    """Compute the sigmoidal function
    Input:
        disances (1D.np.array): vector of distances
        cp (int): central point of the curve
        slope (int): slope of the curve
    Output:
        sig (1D.np.array): the appropriate y-values for distances as x-values
    """
    global ymin
    global ymax

    sig = (ymin + ymax * np.exp((distances - cp) / slope)) / (
        1 + np.exp((distances - cp) / slope)
    )

    return sig


def fitting(distances, RTs):
    """Fit the sigmoidal curve to get the cp and slope

    Input:
        distances (1D.np.array): vector of distances
        RTs (1D.np.array): vector of reaction times

    Output:
        cp (int): central point of the sigmoid
        slope (int): slope of the sigmoid
    """

    global ymin
    global ymax

    # Defines starting points and boundaries for the fitting
    k_0 = (ymax - ymin) / (distances[-1] - distances[0])

    initial_slope = (ymax - ymin) / (4 * k_0)
    middle_distance = np.max(distances) / 2

    init_guess = [middle_distance, initial_slope]
    boundaries = ([0, float("-inf")], [float("inf"), float("inf")])

    # Fits the data
    popt, pcov = curve_fit(
        RTsig,
        xdata=distances,
        ydata=RTs,
        p0=init_guess,
        method="trf",
        ftol=1e-8,
        xtol=1e-8,
        maxfev=10000,
        bounds=boundaries,
    )
    sigpar = np.asarray(popt)

    cp = sigpar[0]
    slope = sigpar[1]

    return cp, slope

In [ ]:
## RT fitting


def fit_RT(xf, yf):
    m = (xf.size * np.sum(xf * yf) - np.sum(xf) * np.sum(yf)) / (
        xf.size * np.sum(xf * xf) - np.sum(xf) ** 2
    )
    bias = (np.sum(yf) - m * np.sum(xf)) / xf.size

    if bias < 0:
        bias = 0
    if m < 0:
        m = 0

    return m * xf + bias

# Spearman-Karber Fitting

### Monotonization Function

In [ ]:
def monotonize(fi, nTrials):
    fiMono = np.copy(fi)

    while not np.all(np.diff(fiMono) >= 0):  # while non monotonic
        i = 0
        while i < len(fiMono) - 1:
            if fiMono[i] <= fiMono[i + 1]:
                i += 1

            else:
                k = 0
                while True:
                    tempfi = np.sum(
                        fiMono[i : i + k + 1] * nTrials[i : i + k + 1]
                    ) / np.sum(nTrials[i : i + k + 1])
                    if i + k > len(fiMono):
                        break
                    elif fiMono[i + k] > tempfi:
                        break
                    else:
                        k += 1

                fiMono[i : i + k + 1] = np.ones(k + 1) * tempfi
                i = i + k + 1

    return fiMono

In [ ]:
def fise(fi):
    fi = (fi - np.min(fi)) / (np.max(fi) - np.min(fi))
    return fi

In [ ]:
def SK(fi, dist, nTrials):
    DLc = 0.6745
    # 75th percentile of standard normal
    s = 500

    fis = fise(fi)  # turn into probs
    fim = monotonize(fis, nTrials)
    fi = np.append(0, fim)
    fi = np.append(fi, 1)
    fi = np.diff(fi)

    distsq = np.square(dist)
    distcube = np.power(dist, 3)

    PSE = 1 / 2 * np.sum(fi * np.diff(distsq) / np.diff(dist))
    M = 1 / 3 * np.sum(fi * np.diff(distcube) / np.diff(dist))
    sigSK = np.sqrt(M - PSE**2)
    DL = sigSK * DLc
    CE = PSE - s

    return PSE, M, sigSK, DL, CE, fim, fis

In [ ]:
# 1359 PSE pre, 504 DL pre play around
# fi = [283.467, 283.761, 303.932, 325.895, 329.403]
#
# s_dist= [3200, 2700, 2200, 1500, 800, 300, -200]
# a_dist = [126, 111, 96, 75, 54, 39, 24]
# SK(fi, a_dist, [1,1,1,1,1])

# Behavioural Parameters From Ferroni et al., 2022 and Di Cosmo et al., 2018

In [ ]:
# Ferroni et al., 2022
# Sigmoidal Parameters

##Pre-Training
###Healthy Controls
fprHCCP = 120 - 1.377 * 30
fprHCS = 1 / 0.11
###Schizophrenia
fprSCZCP = 120 - 1.666 * 30
fprSCZS = 1 / 0.075

##Post Training
###Healthy Controls
fpostHCCP = 120 - 1.028 * 30
fpostHCS = 1 / 0.18
###Schizophrenia
fpostSCZCP = 120 - 1.361 * 30
fpostSCZS = 1 / 0.061


# Spearman-Kaerber Parameters

##Pre-Training
###Healthy Controls
fprHCPSE = 120 - 1.359 * 30
fprHCDL = ...  # 504
###Schizophrenia
fprSCZPSE = 120 - 1.504 * 30
fprSCZDL = ...  # 696

##Post Training
###Healthy Controls
fpostHCPSE = 120 - 1.254 * 30
fpostHCDL = ...  # 540
###Schizophrenia
fpostSCZPSE = 120 - 1.405 * 30
fpostSCZDL = ...  # 613

# Di Cosmo et al., 2018

dcSCZCP = 120 - 1.654 * 30
dcSCZS = 1 / 0.23

In [ ]:
fpreHCRTSig = [279.747, 277.283, 301.396, 326, 331]
import numpy as np

fpostHCRTSig = [
    334.900769230769,
    304.051538461538,
    283.203846153846,
    277.339230769231,
    277.027692307692,
][::-1]

fpreSRTSig = [
    402.582380952381,
    409.355714285714,
    391.851904761905,
    375.072380952381,
    366.409523809524,
][::-1]
fpostSRTSig = [
    425.620952380952,
    404.887142857143,
    378.611428571428,
    368.370952380952,
    384.501904761905,
][::-1]

fpreHCRTSK = [283, 329, 337, 304, 284][::-1]
fpreHCRTSK = (fpreHCRTSK - np.min(fpreHCRTSK)) / (
    np.max(fpreHCRTSK) - np.min(fpreHCRTSK)
)
fpreSRTSK = [
    380.642229577875,
    397.821818801943,
    412.798365471543,
    394.017373770684,
    391.223527385707,
][::-1]
fpreSRTSK = (fpreSRTSK - np.min(fpreSRTSK)) / (np.max(fpreSRTSK) - np.min(fpreSRTSK))


fpostHCRTSK = [280, 336, 313, 291, 280][::-1]
fpostHCRTSK = (fpostHCRTSK - np.min(fpostHCRTSK)) / (
    np.max(fpostHCRTSK) - np.min(fpostHCRTSK)
)
fpostSRTSK = [
    389.654637856702,
    424.157148212331,
    401.931595742975,
    384.72284079883,
    383.285761190903,
][::-1]
fpostSRTSK = (fpostSRTSK - np.min(fpostSRTSK)) / (
    np.max(fpostSRTSK) - np.min(fpostSRTSK)
)

Bibliography:

Paredes, R., Ferri, F., & Seriès, P. (2022). Influence of E/I balance and pruning in peri-personal space differences in schizophrenia: a computational approach. Schizophrenia Research, 248, 368-377.

Serino, A., Canzoneri, E., Marzolla, M., Di Pellegrino, G., & Magosso, E. (2015). Extending peripersonal space representation without tool-use: evidence from a combined behavioral-computational approach. Frontiers in behavioral neuroscience, 9, 4.

Magosso, E., Ursino, M., di Pellegrino, G., Làdavas, E., & Serino, A. (2010). Neural bases of peri-hand space plasticity through tool-use: Insights from a combined computational–experimental approach. Neuropsychologia, 48(3), 812-830.